In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image
from PIL import ImageOps
import random
import math

In [ ]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [ ]:
def rotatedRectWithMaxArea(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle (maximal area) within the rotated rectangle.
    """
    if w <= 0 or h <= 0:
        return 0,0

    width_is_longer = w >= h
    side_long, side_short = (w,h) if width_is_longer else (h,w)

    # since the solutions for angle, -angle and 180-angle are all the same,
    # if suffices to look at the first quadrant and the absolute values of sin,cos:
    sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
    if side_short <= 2.*sin_a*cos_a*side_long or abs(sin_a-cos_a) < 1e-10:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
    else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

    return wr,hr

In [ ]:
label_num = 100

class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 1000 * 1000

    def __getitem__(self, index):
        # print(index)
        label = index % label_num
        img_count = int(index / label_num) % 1000

        img = Image.open(f'{file_path}/train/{label}/{label}_{0 + img_count}.jpg')
        rand = random.randint(0, 11)
        
        img = img.rotate(rand, expand=False)
        new_width, new_height = rotatedRectWithMaxArea(64,64,rand / 180 * math.pi)
        new_width = max(new_width, 56)
        new_height = max(new_height, 56)
        
        crop_area = ((img.size[0]-new_width)/2, (img.size[1]-new_height)/2, (img.size[0]+new_width)/2, (img.size[1]+new_height)/2)
        
        img = img.crop(crop_area)
        
        rand = random.randint(0, 1)
        
        if rand == 0:
            img = ImageOps.mirror(img)
            
        randx = random.randint(0, img.size[0] - 56)
        randy = random.randint(0, img.size[1] - 56)
        
        img = img.crop((randx, randy, 56+randx, 56+randy))
        
        img = my_transform(img)
        
        return img, label

    def __len__(self):
        return self.length

In [ ]:
train_loader = data.DataLoader(ImageSet(), batch_size=256, shuffle=True)
test_loader = data.DataLoader(ImageSet(), batch_size=1, shuffle=True)

In [ ]:
convNet = models.resnet18()
convNet.conv1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False))
convNet.maxpool = nn.Sequential()
# convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(512,label_num)
print(convNet)

In [ ]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = models.resnet18()
convNet.conv1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False))
convNet.maxpool = nn.Sequential()
# convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(512,label_num)
convNet.load_state_dict(torch.load('./ConvNet.model'))
convNet.eval()

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.003)


In [ ]:
convNet.train()
device = torch.device("cuda")
convNet = convNet.to(device)
for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = convNet(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    if i % 10 == 0:
        print(f"i = {i},  loss = {loss},  accuracy = {float(sum(labels == predict))/float(labels.size(0))}")

In [ ]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')

In [ ]:
import moxing as mox
mox.file.copy_parallel("ConvNet.model","obs://deep-learning-hw2-zzzzzzjjjbbb/ConvNet.model")